In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
url='https://rollcall.com/factbase/trump/transcript/donald-trump-speech-campaign-rally-martinsburg-pennsylvania-october-26-2020'
resp=requests.get(url)
soup=BeautifulSoup(resp.text,'html.parser')
print(soup)

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://rollcall.com/app/themes/rollcall/public/favicon.dfdc32.ico" rel="icon" type="image/x-icon"/>
<meta content="#2F3B4B" name="msapplication-TileColor"/>
<link href="https://rollcall.com/app/themes/rollcall/public/pwa-manifest.json" rel="manifest"/>
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots">
<style>img:is([sizes="auto" i], [sizes^="auto," i]) { contain-intrinsic-size: 3000px 1500px }</style>
<meta content="yes" name="mobile-web-app-capable"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="black" name="apple-mobile-web-app-status-bar-style"/>
<meta content="#2F3B4B" name="theme-color"/>
<meta content="#2F3B4B" name="msapplication-TileColor"/>
<meta content="https://rollcall.com/app/themes/rollcall/app/uploads/2022/02/cropped-rollcall-placeholder-

Title

In [3]:
speech_title=soup.find(class_="text-[#2F3C4B] text-center text-xl sm:text-2xl not-italic font-semibold leading-normal sm:leading-9 font-graphik")
speech_titles=speech_title.get_text(strip=True)
speech_title=re.search(r"^(.+)-",speech_titles).group(1).strip()
print(speech_titles)
print(speech_title)
speech_date=re.search(r"-\s(.+)",speech_titles).group(1).strip()
speech_date


Speech: JD Vance Discusses Tax Policy and Public Safety in Corcord, North Carolina - September 24, 2025
Speech: JD Vance Discusses Tax Policy and Public Safety in Corcord, North Carolina


'September 24, 2025'

Number of sentences and number of words, all in one, strong

In [4]:
blocks_container=soup.find(class_="flex flex-wrap gap-8 justify-between")
blocks=blocks_container.find_all(class_='flex-1 h-content')
for block in blocks:
    name_div=block.find(class_="font-graphik text-sm font-medium leading-normal flex items-center").get_text(strip=True)
    if "Trump" in name_div:
        trump_block=block
    
contents=trump_block.find_all(class_="font-graphik text-xs font-medium text-[#2F3C4B]")
contents=[content.get_text(strip=True) for content in contents]
contents
for content in contents :
    if "sentences" in content:
        nbr_sentences=re.search(r'\d+',content).group()
    elif "words" in content:
        nbr_words=re.search(r'\d+',content).group()
    else :
        nbr_seconds=re.search(r'\d+',content).group()
print(nbr_sentences,nbr_words,nbr_seconds)

NameError: name 'trump_block' is not defined

Categories

In [ ]:
categories=soup.find_all(class_="text-[#015582] text-sm font-normal leading-normal rounded-md bg-[#F4F4F5] border border-[#D9D9D9] p-2")
categories=[category.get_text(strip=True) for category in categories]
categories



Cleaning Categories

In [11]:
categories=[category.split('>') for category in categories]
categories=list(set([category[i].strip() for category in categories for i in range(len(category))]))
categories

['Law Enforcement',
 'Police',
 'Immigration',
 'Society',
 'Politics',
 'Crime, Law and Justice',
 'Government',
 'Crime']

Transcriptions

In [23]:
transcriptions=soup.find_all(class_='flex gap-4 py-2')
list_transcriptions=[]
for transcription in transcriptions :
    speaker=transcription.find(class_="text-md inline").get_text(strip=True)
    timestamp=transcription.find(class_='text-xs text-gray-600 inline ml-2').get_text(strip=True)
    text=transcription.find(class_='flex-auto text-md text-gray-600 leading-loose').get_text(strip=True)
    list_transcriptions.append([speaker,timestamp,text])
list_transcriptions

[['Donald Trump',
  '00:00:00-00:00:32 (32 sec)',
  "Thank you so much, everybody. And Sarah, thank you very much. She is -- is she great. How good is she? She's a special -- oh, there's a lot of people here. Thank you very much, Sarah. We landed. That snow is pouring in. It was so beautiful. In fact, I thought we were landing in Alaska. I was going to tell Sarah."],
 ['Donald Trump',
  '00:00:32-00:00:50 (18 sec)',
  "We've got you home in Alaska. This is beautiful and I want to thank everybody, such an incredible turnout. We've had really amazing turnouts. And we're doing so well and we're so proud of it. And we love Wisconsin, it's a special place, and I think we're going to do really well. I think we're going to do really well, really well."],
 ['Donald Trump',
  '00:00:50-00:01:30 (40 sec)',
  "Look at these young people. You know something I said on the way up and I was saying it before that I'm self-funding my campaign. I'm putting up my own money. I don't know if it's worth it.

trump_transcriptions

In [17]:
trump_transcriptions=[transcription_list[1:] for transcription_list in list_transcriptions if "Donald Trump" in transcription_list[0]]
trump_transcriptions


[['00:00:00-00:00:31 (31 sec)',
  "Thank you very much, everybody. This is an honor. What a crowd this is. Look how that crowd goes. This is a big crowd. A very special hello to Wisconsin. We really appreciate it. We've had great success here. We've had tremendous success, and I'm thrilled to be back in this incredible state with the thousands of proud, hardworking American patriots, which is what you are."],
 ['00:00:31-00:00:51 (21 sec)',
  "And if you can believe it, 59 days from now, we are going to win Wisconsin. We are going to defeat Comrade Kamala Harris, and we are going to win the White House, that gorgeous, beautiful White House. We're going to win it, and we're going to turn this country around because this country is a failing nation right now."],
 ['00:00:51-00:01:13 (21 sec)',
  "It's a laughingstock all over the world. Over the past four years, our country has seen the sickness and corruption of the Washington swamp exposed like never before. You've seen it. You've been

Création de la première row de la DB

In [10]:
import pandas as pd

df=pd.DataFrame(columns=["Date","Title","Categories","Nbr Sentences","Nbr Words", "Nbr Seconds", "Transcriptions_list"])
df.loc[len(df)]=[speech_date,speech_title,categories,nbr_sentences,nbr_words,nbr_seconds,trump_transcriptions]
df

,Date,Title,Categories,Nbr Sentences,Nbr Words,Nbr Seconds,Transcriptions_list
0,"September 8, 2025",Speech: Donald Trump Addresses the Religious L...,"[Court, Belief Systems, Politics, Judiciary, C...",703,7094,2588,"[[00:00:00-00:00:22 (23 sec), Well, thank you ..."


In [14]:
df.iloc[0].loc['Transcriptions_list']

[['00:00:00-00:00:22 (23 sec)',
  "Well, thank you very much. Thank you very much. I said we'll have to cut Lee Greenwood a little bit short. How good is he? How good is he, right? He's been with us right from the beginning and I wanted to stand and listen to the entire song. But people would have said, look at the ego of that guy, he just wouldn't stop. [Laughter] He just stood there and listened to the applause."],
 ['00:00:22-00:00:46 (24 sec)',
  'But I appreciate it very much, and I want to thank you for doing an incredible job, Pam Bondi. The attorney general has been unbelievable. Incredible job. Thank you, Pam. Amazing. And thank you to all of the faith leaders, pastors and patriots who have come to the Museum of the Bible for this historic meeting of the Presidential Commission on Religious Liberty.'],
 ['00:00:46-00:01:10 (24 sec)',
  "These are incredible people, and everybody wanted to be on this commission, and I made a couple of people unhappy -- maybe even enemies for li